<a href="https://colab.research.google.com/github/edgarreis/Jarvis-Gemini-Assistant/blob/main/Jarvis_Gemini_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Jarvis: Um assistente de voz utilizando Google GenerativeAI

Projetos de Referência



*   Gemini API Python quickstart
https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/quickstart_colab.ipynb

*   Upload an audio file with the File API
https://github.com/google-gemini/cookbook/blob/main/quickstarts/Audio.ipynb
*   Text to Speech
https://github.com/shuvobasak4004/Text-to-Voice/blob/main/TexttoV.ipynb

## Instalações do SDK e demais Libs

In [1]:
!pip install -q -U google-generativeai

In [2]:
!pip install gtts

In [3]:
!pip install playsound

## Bibliotecas

In [4]:
#IO
from IPython.display import HTML, Javascript, clear_output, Audio, display
from google.colab.output import eval_js
#Audio
from gtts import gTTS
from google.colab import output
from playsound import playsound
#config
import base64
import time

## Conversão de Audio para .WAV (Javascript)

Função para Gravar o Audio

In [5]:
# Código Python para exibir botão de gravação e salvar áudio

js = Javascript('''
  // Código JavaScript para gravação de áudio

  async function recordAudio() {
    const div = document.createElement('div');
    const strtButton = document.createElement('button');
    const stopButton = document.createElement('button');

    // Estilos para os botões
    // Botão maior para pessoas com baixa visão
    const buttonStyle = 'font-size: 40px; padding: 20px;';  // Define o tamanho da fonte e o espaçamento interno

    strtButton.textContent = 'Clique para Gravar';
    strtButton.style = buttonStyle;  // Aplica os estilos ao botão
    stopButton.textContent = 'Enviar';
    stopButton.style = buttonStyle;  // Aplica os estilos ao botão

    document.body.appendChild(div);
    div.appendChild(strtButton);

    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    let recorder = new MediaRecorder(stream);
    let audio = new Audio();

    audio.style.display = 'block';
    audio.controls = true;
    audio.muted = true;

    await new Promise((resolve) => strtButton.onclick = resolve);
    strtButton.replaceWith(stopButton);
    recorder.start();

    await new Promise((resolve) => stopButton.onclick = resolve);
    recorder.stop();

    let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
    let arrBuff = await recData.data.arrayBuffer();
    let audioTracks = stream.getAudioTracks()[0];
    div.remove();

    let binaryString = '';
    let bytes = new Uint8Array(arrBuff);
    bytes.forEach((byte) => { binaryString += String.fromCharCode(byte); });

    const url = URL.createObjectURL(recData.data);
    const player = document.createElement('audio');
    player.controls = false;
    player.src = url;
    document.body.appendChild(player);

    // Remove o elemento de áudio após a gravação
    player.addEventListener('ended', () => {
      player.remove();
    });

    return btoa(binaryString);
  }
''')


##Conversão de Texto para Audio

Função de Conversão de Texto para Fala

In [6]:

def text_to_speech(text):
    #text = input("Enter the text to convert to speech (Portugues): ")
    language = 'pt'  # Set the language code to 'br' for Portugues

    tts = gTTS(text=text, lang=language)
    tts.save('output.mp3')

    audio = Audio('output.mp3', autoplay=True)
    output.clear()
    display(audio)


##Gemini API Python

##Set up your API key

In [7]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

##Initialize the Generative Model

Configuração do modelo

In [8]:
# Configuraçãoes Gerais
generation_config = {
    "candidate_count": 1,
    "temperature":0.5,
}

# Configurações de Segurança
safety_settings = {
    "HARASSMENT": "BLOCK_MEDIUM_AND_ABOVE",
    "HATE": "BLOCK_MEDIUM_AND_ABOVE",
    "SEXUAL": "BLOCK_MEDIUM_AND_ABOVE",
    "DANGEROUS": "BLOCK_MEDIUM_AND_ABOVE"
}

Inicializando o Modelo Gemini

In [9]:
model = genai.GenerativeModel(
    model_name = "gemini-1.5-pro-latest",
    generation_config = generation_config,
    safety_settings = safety_settings,
    system_instruction="Considere que você esta falando comm pessoas com deficiência visua, retorne respostas curtas e objetivas"
)

Inicializa um Novo Chat

In [10]:
chat = model.start_chat(history=[])

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

##Upload do arquivo de Audio para a API

Gravação do Audio

In [11]:

def gravacao():
  # Cria Botão de Gravação
  display(js)

  # Grava
  output_recorded = eval_js('recordAudio()')

  # Salva
  with open('audio.wav', 'wb') as file:
    binary = base64.b64decode(output_recorded)
    file.write(binary)

  # Mostra
  #print('Recording saved to:', file.name)


Upload do Audio no Gemini

In [12]:
# audio_pergunta = genai.upload_file(path='/content/audio.wav')

##Utiliza o arquino no prompt

In [13]:
def audio_prompt():
  # Upload do Audio no Gemini
  audio_pergunta = genai.upload_file(path='/content/audio.wav')
  # Montando o prompt
  response = chat.send_message(audio_pergunta)
  '''
  # Montando o prompt
  prompt = "Responda de forma objetiva em uma linha: "
  model = genai.GenerativeModel('models/gemini-1.5-pro-latest')
  response = model.generate_content([prompt, audio_pergunta])
  '''
  # Convertendo para texto
  resposta = str(response.text)
  return resposta

##LOOP PARA CONVERSA COM O CHAT

In [ ]:
resposta_jarvis = ''

while resposta_jarvis != 'fim':
  #clear_output(wait=True)
  # Gravação do Audio
  gravacao()
  # Envio do Audio para o Gemini e sua resposta em Texto
  resposta_jarvis = audio_prompt()
  # Fala do Texto via Google Text-to-Speech
  text_to_speech(resposta_jarvis)
